In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import pickle
import joblib

In [ ]:
df = pd.read_parquet('../data/all_data_preprocessed/all_merged.parquet')

In [ ]:
X = df.drop(columns=[
    'event_all_region','alarms_in_regions',"event_lastDay_region",
    'event_1h_ago',"event_lastDay_region"])
y = df['event_all_region']

In [ ]:
X.rename(columns={"event_2h_ago": "status"}, inplace=True)

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
splits = list(tscv.split(X))
train_idx, test_idx = splits[-1]
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    eval_metric='aucpr',
)


In [ ]:
xgb_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = xgb_model.predict(X_test_scaled)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
print(
classification_report(y_test, y_pred))

In [ ]:
importance = xgb_model.feature_importances_
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})

In [ ]:
importance_df = importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
top_20_features = importance_df.head(20)
print(top_20_features)

In [ ]:
with open('../src/our_models/3_Xgboost_2hour.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)

In [ ]:
X_train

In [ ]:
X

In [ ]:
joblib.dump(scaler, '../src/our_models/scaler_2hour.pkl')